# Обучение модели

In [ ]:
import tensorflow
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization, Activation, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import pandas as pd
import numpy as np
 

In [ ]:
def load_train(path):
 
    labels = pd.read_csv(path + 'labels.csv')
    images_dir = path + '/final_files'
 
    data_gen = ImageDataGenerator(rescale=1./255., validation_split=0.25)
 
    data_gen_flow = data_gen.flow_from_dataframe(dataframe=labels, directory=images_dir,
                                                x_col='file_name', y_col='real_age',
                                                target_size=(256,256), batch_size=32,
                                                class_mode='raw', seed=1337, subset='training')
 
    return data_gen_flow

In [ ]:
def load_test(path):
    labels_test = pd.read_csv(path+'labels.csv')
    test_datagen = ImageDataGenerator(rescale=1/255., validation_split=0.25)
    
    test_datagen_flow = test_datagen.flow_from_dataframe(dataframe=labels_test, directory=path+'/final_files',
                                                         x_col='file_name', y_col='real_age',
                                                         target_size=(224,224), batch_size=32,
                                                         class_mode='raw', subset='validation', seed = 12345)
    
    return test_datagen_flow

In [ ]:
def create_model(input_shape):
    
    backbone = ResNet50(input_shape=input_shape, 
                        weights='imagenet',
                        include_top=False)
    
    optimizer = Adam(lr=0.0001)
    
    model = Sequential()
    model.add(backbone)
    model.add(GlobalAveragePooling2D())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(12, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='relu'))
    
    model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mae'])

    return model 

In [ ]:
def train_model(model, train_data, test_data, batch_size=None, epochs=10, steps_per_epoch=None, validation_steps=None):
    
    if steps_per_epoch is None:
        steps_per_epoch = len(train_data)
    if validation_steps is None:
        validation_steps = len(test_data)
   
    model.fit(train_data,
              validation_data=test_data,
              batch_size=batch_size, 
              epochs=epochs,
              steps_per_epoch=steps_per_epoch,
              validation_steps=validation_steps,
              verbose=2, shuffle=True)
    
    return model